# Notebook 11: Create a Table for running Pstools<a class="tocSkip">


In [1]:
%%capture
%pip install pyfaidx

In [2]:
%%capture
%pip install gcsfs

In [3]:
%%capture
%pip install --upgrade --no-cache-dir terra-pandas
%pip install --upgrade --no-cache-dir terra-notebook-utils

In [4]:
%%capture
%pip install --no-cache-dir -U crcmod

# Import Statements & Global Variable Definitions

In [5]:
%%capture 
import os
import io
import pandas as pd
import numpy as np
import gcsfs
import gzip

import terra_notebook_utils as tnu
import terra_pandas as tp

## Global Variable Definitions

## Set Environment Variables

In [6]:
# Get the Google billing project name and workspace name
PROJECT = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.path.basename(os.path.dirname(os.getcwd()))
bucket = os.environ['WORKSPACE_BUCKET'] + "/"

# Verify that we've captured the environment variables
print("Billing project: " + PROJECT)
print("Workspace: " + WORKSPACE)
print("Workspace storage bucket: " + bucket)

Billing project: human-pangenome-ucsc
Workspace: HPRC_Reassembly
Workspace storage bucket: gs://fc-0c2122a8-6725-4199-b90e-828ab006078f/


# Read In Data Tables

## Read In Data Table w/ Final Assemblies

In [7]:
final_assemblies_df = sample_df = tp.table_to_dataframe("assembly_sample", workspace="AnVIL_HPRC", workspace_namespace="anvil-datastorage")
final_assemblies_df.head()

,mat_fasta,pat_fasta
assembly_sample_id,,
HG002,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00438,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG005,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00621,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00673,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...


## Read In Sample Data Table

In [8]:
sample_df = tp.table_to_dataframe("sample", workspace="AnVIL_HPRC", workspace_namespace="anvil-datastorage")
sample_df.head()

,mat_ilmn,cohort,hifi,paternal_id,hic,maternal_id,child_ilmn,bionano_cmap,bionano_bnx,nanopore,pat_ilmn,strandseq,notes
sample_id,,,,,,,,,,,,,
HG002,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HPRC_PLUS,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG003,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG004,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,NaN
HG00438,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HPRC,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00436,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00437,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,NaN,NaN
HG00480,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HPRC,NaN,HG00478,NaN,HG00479,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,NaN,NaN,NaN,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,NaN,Abnormal Karyotype
HG005,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HPRC_PLUS,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG006,NaN,HG007,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,NaN,NaN,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,NaN,NaN
HG00621,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HPRC,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00619,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,HG00620,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,NaN,NaN


In [9]:
## Just keep the columns we need
sample_df = sample_df[["hic"]]

# Combine Data Frames
## Add HiC data To Assembly Table

In [10]:
launch_df = sample_df.copy()

In [11]:
launch_df = pd.merge(launch_df, final_assemblies_df, how='left', left_index=True, right_index=True)

In [12]:
launch_df

,hic,mat_fasta,pat_fasta
sample_id,,,
HG002,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00438,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00480,NaN,NaN,NaN
HG005,NaN,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00621,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00673,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00733,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00735,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00741,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...


In [13]:
# Upload Pstools Sample Data Table

In [13]:
launch_df = launch_df.rename_axis("pstools_sample_id")
launch_df.head()

,hic,mat_fasta,pat_fasta
pstools_sample_id,,,
HG002,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00438,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00480,NaN,NaN,NaN
HG005,NaN,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00621,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...


In [19]:
launch_df.dropna(inplace=True)

In [21]:
launch_df

,hic,mat_fasta,pat_fasta
pstools_sample_id,,,
HG002,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00438,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00621,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00673,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00733,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00735,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG00741,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG01071,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...
HG01106,[gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...,gs://fc-4310e737-a388-4a10-8c9e-babe06aaf0cf/w...


In [ ]:
tp.dataframe_to_table("pstools_sample", launch_df)